In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
train_datagen = ImageDataGenerator(rescale = 1./255,rotation_range=15,
                                    width_shift_range=0.15,
                                    height_shift_range=0.15,
                                    shear_range=0.15,
                                    zoom_range=0.15,
                                    horizontal_flip=True,
                                  )
val_datagen = ImageDataGenerator(rescale=1./255)

In [3]:
#Batch Size and Image Size
batch_sz = 64
sz = 48

train_generator = train_datagen.flow_from_directory('../input/fer-dsc-v3/final_data_v5/final_data_v5/Train',
                                                    batch_size = batch_sz,
                                                    class_mode='categorical',
                                                    target_size=(sz,sz))

val_generator = val_datagen.flow_from_directory('../input/fer-dsc-v3/final_data_v5/final_data_v5/Test',
                                                    batch_size = batch_sz,
                                                    class_mode='categorical',
                                                    target_size=(sz,sz))

Found 13651 images belonging to 4 classes.
Found 1649 images belonging to 4 classes.


In [4]:
'''class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>0.74):
#             if(logs.get('accuracy')>78):
                print("\nReached 74% val_accuracy, so cancelling training!")
                self.model.stop_training = True
callbacks = myCallback()'''

'class myCallback(tf.keras.callbacks.Callback):\n    def on_epoch_end(self, epoch, logs={}):\n        if(logs.get(\'val_accuracy\')>0.74):\n#             if(logs.get(\'accuracy\')>78):\n                print("\nReached 74% val_accuracy, so cancelling training!")\n                self.model.stop_training = True\ncallbacks = myCallback()'

In [5]:
from keras.applications.vgg16 import VGG16

In [6]:
pre_model = VGG16(include_top = False,
                 weights = 'imagenet',
                 input_shape = (48,48,3))

58892288/58889256 [==============================] - 0s 0us/step


In [7]:
for layer in pre_model.layers:
    layer.trainable=False

In [8]:
last_output = pre_model.get_layer('block5_conv1').output

In [9]:
x = tf.keras.layers.GlobalAveragePooling2D()(last_output)
x = tf.keras.layers.Flatten()(last_output)
x = tf.keras.layers.Dense(256,activation='relu')(x)
x = tf.keras.layers.Dropout(0.4)(x)
x = tf.keras.layers.Dense(128,activation= 'relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(4,activation='softmax')(x)

In [10]:
model = tf.keras.Model(pre_model.input, x)

In [11]:
opt = tf.keras.optimizers.RMSprop(learning_rate=1e-5)
model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator,
                   validation_data=val_generator,
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                   validation_steps=val_generator.samples//val_generator.batch_size,
                    epochs=30
                    )

Epoch 1/30
213/213 [==============================] - 44s 208ms/step - loss: 1.9499 - accuracy: 0.2619 - val_loss: 1.4088 - val_accuracy: 0.3275
Epoch 2/30
213/213 [==============================] - 26s 122ms/step - loss: 1.5160 - accuracy: 0.2926 - val_loss: 1.3444 - val_accuracy: 0.3681
Epoch 3/30
213/213 [==============================] - 26s 123ms/step - loss: 1.4181 - accuracy: 0.3115 - val_loss: 1.3084 - val_accuracy: 0.3994
Epoch 4/30
213/213 [==============================] - 26s 121ms/step - loss: 1.3835 - accuracy: 0.3269 - val_loss: 1.2961 - val_accuracy: 0.4238
Epoch 5/30
213/213 [==============================] - 26s 123ms/step - loss: 1.3644 - accuracy: 0.3424 - val_loss: 1.2858 - val_accuracy: 0.4300
Epoch 6/30
213/213 [==============================] - 26s 120ms/step - loss: 1.3472 - accuracy: 0.3541 - val_loss: 1.2753 - val_accuracy: 0.4294
Epoch 7/30
213/213 [==============================] - 26s 121ms/step - loss: 1.3341 - accuracy: 0.3665 - val_loss: 1.2655 - val_ac

In [ ]:
for layer in model.layers:
    layer.trainable = True


In [ ]:
print(model.summary())

In [ ]:
opt1 = tf.keras.optimizers.Adam(learning_rate = 1e-4)
model.compile(optimizer = opt1, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history1 = model.fit(train_generator,
                    validation_data = val_generator,
                    validation_steps = val_generator.samples // val_generator.batch_size,
                    epochs = 22)
                    #callbacks = [callbacks])

In [ ]:
N = 22
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), history1.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), history1.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), history1.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), history1.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on FER Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")

In [ ]:
model.save('VGG_16_modified.h5')

In [ ]:
model.save_weights('VGG16_modified_weights.h5')

In [ ]:
from keras.models import model_from_json
# serialize model to json
json_model = model.to_json()
#save the model architecture to JSON file
with open('VGG16_modified_model.json', 'w') as json_file:
    json_file.write(json_model)

In [ ]:
test = '../input/fer-dsc-v3/final_data_v5/final_data_v5/Test'
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory (test, batch_size=64, class_mode='categorical', target_size=(48,48))

In [ ]:
model.evaluate(test_generator, steps=4)

In [ ]:
'''import numpy as np
import files
from keras.preprocessing import image

uploaded = files.upload()

# predicting images
for fn in uploaded.keys():
    path = fn
    img = image.load_img(path, target_size=(48, 48))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
    print(fn)
    print(classes)'''